# Paddle OCR

## Imports

In [ ]:
# !pip install paddlepaddle
# !pip install paddleocr
# !pip install paddlepaddle-gpu

!pip install paddleocr
!python -m pip install paddlepaddle-gpu==2.5.0.post118 -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install paddle2onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 466.1/466.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 97.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.4/148.4 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 45.0 MB/s eta 0:00:00


In [ ]:
import paddleocr
from paddleocr import PaddleOCR
from datetime import datetime
import os
from google.colab.patches import cv2_imshow
import warnings
import numpy as np
from PIL import Image
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/paddle/fluid/framework.py:549: UserWarning: You are using GPU version Paddle, but your CUDA device is not set properly. CPU device will be used by default.
  warnings.warn(


In [ ]:
#ocr = PaddleOCR(lang="en", rec_model_type='en', use_gpu=False, use_fp16=True,  precision='fp16')
# 1.49sec, 1.51

# ocr = PaddleOCR(lang="latin", rec_model_type='latin', use_gpu=False, use_fp16=True, det_algorithm='DB',  precision='fp16', log_level='OFF',
#                 rec_model_path='./models/latin_mobile_lite_v2.0.1.paddle') #1.52 sec

In [ ]:
# ocr = PaddleOCR(lang="en", rec_model_type='en', use_gpu=False, log_level='OFF',
#                 rec_model_path='./models/en_mobile_lite_v2.0.1.paddle', use_fp16=True)


ocr = PaddleOCR(lang="en", rec_model_type='en', rec_algorithm = 'CRNN',
                use_gpu = False, det_db_thresh=0.4, show_log = True, use_angle_cls = True)

# latin -> 1.35 sec, en -> 1.41 sec (en model with better accuracy)
# 1.1 sec (630,300) #show_log = False,

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Utils

In [ ]:
def make_statement(nums):
    unit = 'g'
    for i in range(len(nums)):
        s = str(nums[i])
        if len(s)>3 and '.' not in s:
            nums[i] = int(s[:3])
        if '.' in s:
            ind = s.index('.')
            nums[i] = float(s[:ind+2])
    if nums[-1]<=4:
        unit = 'kg'

    if len(nums)==1:
        return f'{nums[0]} kg'
    if len(nums)==2:
        return f'{nums[0]} Pieces X {nums[1]}{unit}'
    if len(nums)==3:
        if nums[0]==(nums[1]*nums[2])/1000:
            return f'{nums[0]} kg ({nums[1]} Pieces X {nums[2]}{unit})'
        else:
            return f'{nums[0]} Pieces X {nums[1]} Pieces X {nums[2]}{unit}'
    return nums

def get_nums(statement):
    nums = []
    curr = ''
    for i in range(len(statement)):
        if statement[i].isdigit() or (len(curr)>0 and statement[i]=='.'):
            curr+=statement[i]
            if i<len(statement)-1 and statement[i+1]=='.': continue

            if i==len(statement)-1 or not statement[i+1].isdigit():
                if float(curr)==int(float(curr)): nums.append(int(curr))
                else: nums.append(float(curr))
                curr=''
    return nums


def get_target_statement(words):
    for sentence in words:
        if 'X' in sentence or 'x' in sentence or '×' in sentence:
            nums = get_nums(sentence)
            if len(nums)>=2:
                return(sentence)
        if 'kg' in sentence and 'BULK' in sentence:
            return(sentence)
    return None

def load_image_with_orientation(image_path):
    img = Image.open(image_path)

    # Check if the image has Exif data with orientation information
    if hasattr(img, '_getexif'):
        exif = img._getexif()
        if exif is not None:
            orientation = exif.get(0x0112)
            # Orientation values: 1, 3, 6, 8 (See Exif documentation for details)
            if orientation == 3:
                img = img.rotate(180, expand=True)
            elif orientation == 6:
                img = img.rotate(270, expand=True)
            elif orientation == 8:
                img = img.rotate(90, expand=True)

    return img

## Code

In [ ]:
!pip install paddleocr_convert

In [ ]:
from paddleocr_convert import PaddleOCRModelConvert

converter = PaddleOCRModelConvert()
save_dir = '/content/drive/MyDrive/OCR-SKU/Models'
#online
url = 'https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar'
txt_url = 'https://raw.githubusercontent.com/PaddlePaddle/PaddleOCR/release/2.6/ppocr/utils/ppocr_keys_v1.txt'

converter(url, save_dir, txt_path=txt_url)

Downloading: 100%|██████████| 2.19M/2.19M [00:00<00:00, 4.93MiB/s]


Successfully convert model to /content/drive/MyDrive/OCR-SKU/Models/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.onnx
The model has changed to dynamic inputs.


PosixPath('/content/drive/MyDrive/OCR-SKU/Models/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.onnx')

In [62]:
ocr = PaddleOCR(lang="en", use_onnx = True, show_log = True, use_gpu = False,
                rec_model_dir = f'{path1}en_PP-OCRv3_rec_infer/model.onnx',
                cls_model_dir = f'{path1}ch_ppocr_mobile_v2.0_cls_infer/model.onnx', det_model_dir =f'{path1}en_PP-OCRv3_det_infer/model.onnx')

[2023/08/22 11:06:27] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_det_infer/model.onnx', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_rec_infer/model.onnx', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6

In [ ]:
os.listdir('/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_rec_infer/')

['inference.pdmodel',
 'inference.pdiparams.info',
 'inference.pdiparams',
 'en_PP-OCRv3_rec_infer.onnx',
 'model.onnx']

In [ ]:
!paddle2onnx --model_dir '/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_rec_infer/' \
             --model_filename inference.pdmodel \
             --params_filename inference.pdiparams\
             --save_file '/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_det_infer/model.onnx' \
             --input_shape_dict="{'x':[-1,3,-1,-1]}" \
             --enable_dev_version True

In [ ]:
import onnxruntime
det_model = onnxruntime.InferenceSession("/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_det_infer/model.onnx")
rec_model = onnxruntime.InferenceSession("/content/drive/MyDrive/OCR-SKU/Models/en_PP-OCRv3_rec_infer/model.onnx")
cls_model = onnxruntime.InferenceSession("/content/drive/MyDrive/OCR-SKU/Models/ch_ppocr_mobile_v2.0_cls_infer/model.onnx")

In [ ]:
img = np.array(Image.open("/content/IMG-20230708-WA0009.jpg"))
img = img.astype(np.float32) / 255.0
img_tensor = img.transpose([2, 0, 1])
img_tensor = np.expand_dims(img_tensor, axis=0)

#print(img.shape)
out = det_model.run(None, {"x": img_tensor})

In [ ]:
out[0].shape

(1, 1, 1280, 576)

In [ ]:
rec_outputs = rec_model.run(None, {"x": img_tensor})

InvalidArgument: ignored

In [ ]:
my_path = "/content/drive/MyDrive/OCR-SKU/Amul_Butter_targetted/"
times = []
for f in sorted(os.listdir(my_path)):
    # Read the image file.
    image_path = my_path+f

    t1 = datetime.now()
    img = load_image_with_orientation(image_path)
    img = np.array(img.resize((48, 48)))
    print(img.shape)
    #img = img[:,100:700]

    #result = ocr.ocr(image_path)
    result = ocr.ocr(img)
    words = []
    for line in result:
        for word_info in line:
            words.append(word_info[1][0])

    req = get_target_statement(words)
    if req:
        nums = get_nums(req)
        print(make_statement(nums))
    else: print(f)
    t2 = datetime.now()
    times.append(t2-t1)

In [ ]:
sorted(times)[-2]

datetime.timedelta(seconds=3, microseconds=789311)

## Data and Utils

In [ ]:
!unzip "/content/drive/MyDrive/OCR-SKU/Box Counting Details.zip"

Archive:  /content/drive/MyDrive/OCR-SKU/Box Counting Details.zip
   creating: Box Counting Details/Amul Butter/
   creating: Box Counting Details/Amul Butter/1 Amul Pasteurized butter salted 20-500g/
  inflating: Box Counting Details/Amul Butter/1 Amul Pasteurized butter salted 20-500g/IMG-20230708-WA0007.jpg  
  inflating: Box Counting Details/Amul Butter/1 Amul Pasteurized butter salted 20-500g/IMG-20230708-WA0008.jpg  
  inflating: Box Counting Details/Amul Butter/1 Amul Pasteurized butter salted 20-500g/IMG-20230708-WA0009.jpg  
  inflating: Box Counting Details/Amul Butter/1 Amul Pasteurized butter salted 20-500g/IMG-20230708-WA0013.jpg  
   creating: Box Counting Details/Amul Butter/10 Amul Pasteurized table butter 8u-8u-200g/
  inflating: Box Counting Details/Amul Butter/10 Amul Pasteurized table butter 8u-8u-200g/IMG-20230708-WA0067.jpg  
  inflating: Box Counting Details/Amul Butter/10 Amul Pasteurized table butter 8u-8u-200g/IMG-20230708-WA0068.jpg  
  inflating: Box Countin

In [ ]:
def make_statement(nums):
    unit = 'g'
    if nums[-1]<4:
        unit = 'kg'
    for i in range(len(nums)):
        s = str(nums[i])
        if len(s)>3 and '.' not in s:
            nums[i] = int(s[:3])
        if '.' in s:
            ind = s.index('.')
            nums[i] = float(s[:ind+2])
        if s[-1]=='9':
            if len(s)==1:
                nums.pop(i)
            else:
                nums[i] = float(s[:-1])
    if len(nums)==1:
        return f'{nums[0]} Pieces'
    if len(nums)==2:
        return f'{nums[0]} Pieces X {nums[1]}{unit}'
    if len(nums)==3:
        return f'{nums[0]} Pieces ({nums[1]} Pieces X {nums[2]}{unit})'
    return nums

def get_nums(statement):
    nums = []
    curr = ''
    for i in range(len(statement)):
        if statement[i].isdigit() or (len(curr)>0 and statement[i]=='.'):
            curr+=statement[i]
            if i<len(statement)-1 and statement[i+1]=='.': continue

            if i==len(statement)-1 or not statement[i+1].isdigit():
                if float(curr)==int(float(curr)): nums.append(int(curr))
                else: nums.append(float(curr))
                curr=''
    return nums


def get_target_statement(words):
    req = ''
    for sentence in words:
        if ('X' in sentence or 'x' in sentence) and 'Pieces' in sentence:
            nums = get_nums(sentence)
            if len(nums)>=2:
                req = sentence
    if len(req)>2:
        return(req)
    return None

def load_image_with_orientation(image_path):
    img = Image.open(image_path)

    # Check if the image has Exif data with orientation information
    if hasattr(img, '_getexif'):
        exif = img._getexif()
        if exif is not None:
            orientation = exif.get(0x0112)
            # Orientation values: 1, 3, 6, 8 (See Exif documentation for details)
            if orientation == 3:
                img = img.rotate(180, expand=True)
            elif orientation == 6:
                img = img.rotate(270, expand=True)
            elif orientation == 8:
                img = img.rotate(90, expand=True)

    return img

## Code

In [ ]:
ocr = PaddleOCR(lang="en", rec_model_type='en', rec_algorithm = 'CRNN', det_db_thresh=0.4, show_log = False, use_angle_cls = True)

In [ ]:
import cv2

In [ ]:
my_path = '/content/Box Counting Details/Amul Butter/'
files = sorted(os.listdir(my_path))
times = []
f = "9 Amul Pasteurized table butter 40pc-500g"

for f in files:
    if '.xlsx' in f: continue
    print('\n',f)
    for image in os.listdir(my_path+f):
        if image[-3:]=='jpg' or image[-3:]=='png':
            # Read the image file.
            image_path = f"{my_path}{f}/{image}"

            t1 = datetime.now()
            # img = load_image_with_orientation(image_path)
            # img = np.array(img.resize((800, 300)))
            # img = img[:,100:700]
            result = ocr.ocr(image_path)
            #result = ocr.ocr(img)
            words = []
            for line in result:
                for word_info in line:
                    words.append(word_info[1][0])
            #print(words)
            req = get_target_statement(words)
            if req:
                nums = get_nums(req)
                print('\t',make_statement(nums))
            t2 = datetime.now()
            times.append(t2-t1)


 1 Amul Pasteurized butter salted 20-500g
	 20 Pieces X 500g

 10 Amul Pasteurized table butter 8u-8u-200g
	 8 Pieces X 8 Pieces X 200g
	 8 Pieces X 200g

 11 Amul White butter Unsalted 150pc-100g
	 150 Pieces X 100g

 12 Amul Pasteurized butter 30pc-500g
	 30 Pieces X 500g
	 30 Pieces X 500g

 13 Amul White butter 15kg

 14 Amul Pasteurized butter 150pc-100g
	 150 Pieces X 100g
	 150 Pieces X 100g

 15 Amul White butter 30pc-500g
	 30 Pieces X 500g

 16 Amul Pasteurized TABLE butter 15kg

 17 Amul Pasteurized butter 60pc-100g
	 60 Pieces X 100g

 18 Amul Pasteurized TABLE butter 10pc-1kg
	 10 Pieces X 1kg
	 10 Pieces X 1kg

 19 Amul Pasteurized butter 10u-1kg
	 10 Pieces X 1kg
	 10 Pieces X 1kg

 2 Amul Pasteurized butter salted 100-100g
	 100 Pieces X 100g

 20 Amul Butter Tinned 36pc-400g
	 36 Pieces X 400g

 21 Amul Pasteurized butter salted 10-100-8g
	 36 Pieces X 400g
	 10 Pieces X 100 Pieces X 8g
	 36 Pieces X 400g

 22 Amul Pasteurized Butter 36u-400gT
	 36 Pieces X 400g
	 36 

In [ ]:
times = np.array(times)
print('Average Time',np.mean(times))

Average Time 0:00:00.505333


In [ ]:
np.median(times), max(times)

(datetime.timedelta(microseconds=498582),
 datetime.timedelta(microseconds=561678))

In [ ]:
my_path = '/content/Box Counting Details/Amul Powder/'
files = sorted(os.listdir(my_path))
times = []
f = '6 Amul Spray IMF IMS - 2pc (420pc 13.5grm)'
for f in files:
    if '.xlsx' in f: continue
    print('\n',f)
    for image in os.listdir(my_path+f):
        if image[-3:]=='jpg' or image[-3:]=='png':
            # Read the image file.
            image_path = f"{my_path}{f}/{image}"

            t1 = datetime.now()
            run_function_with_timeout(image_path, timeout_sec=1.8)
            t2 = datetime.now()
            times.append(t2-t1)


 1 Amul Spray IMF IMS - 12pc 1kg
[2023/07/22 10:31:22] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 12 Pieces X 1kg
[2023/07/22 10:31:23] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 12 Pieces X 1kg
[2023/07/22 10:31:25] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


Process Process-15:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/timeout_decorator/timeout_decorator.py", line 82, in new_function
    return function(*args, **kwargs)
  File "<ipython-input-31-864ac19e3ab7>", line 6, in execute_function_and_get_output
    output = my_func(path)
  File "<ipython-input-31-864ac19e3ab7>", line 23, in my_func
    result = ocr.ocr(img)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/paddleocr.py", line 555, in ocr
    dt_boxes, rec_res, _ = self.__call__(img, cls)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infer/predict_system.py", line 95, in __call__
    rec_res, elapse = self.text_recognizer(img_crop_list)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infe

[2023/07/22 10:31:28] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


Process Process-16:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/timeout_decorator/timeout_decorator.py", line 82, in new_function
    return function(*args, **kwargs)
  File "<ipython-input-31-864ac19e3ab7>", line 6, in execute_function_and_get_output
    output = my_func(path)
  File "<ipython-input-31-864ac19e3ab7>", line 23, in my_func
    result = ocr.ocr(img)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/paddleocr.py", line 555, in ocr
    dt_boxes, rec_res, _ = self.__call__(img, cls)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infer/predict_system.py", line 95, in __call__
    rec_res, elapse = self.text_recognizer(img_crop_list)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infe


 10 Amulya Dairy Whitener - AR AP - 2pc (420pc 10g)
[2023/07/22 10:31:31] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 2 Pieces (420 Pieces X 10g)
[2023/07/22 10:31:32] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 2 Pieces (420 Pieces X 10g)
[2023/07/22 10:31:33] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


...

[2023/07/22 10:32:25] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


Process Process-43:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/timeout_decorator/timeout_decorator.py", line 82, in new_function
    return function(*args, **kwargs)
  File "<ipython-input-31-864ac19e3ab7>", line 6, in execute_function_and_get_output
    output = my_func(path)
  File "<ipython-input-31-864ac19e3ab7>", line 23, in my_func
    result = ocr.ocr(img)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/paddleocr.py", line 555, in ocr
    dt_boxes, rec_res, _ = self.__call__(img, cls)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infer/predict_system.py", line 95, in __call__
    rec_res, elapse = self.text_recognizer(img_crop_list)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infe

[2023/07/22 10:32:27] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
8 Amulya Dairy Whitener - AR AP - 24pc 500g

 9 Amulya Dairy Whitener - AR AP - 2pc (240pc 21g)
[2023/07/22 10:32:29] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 2 Pieces (240 Pieces X 21g)
[2023/07/22 10:32:31] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process


Process Process-46:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/timeout_decorator/timeout_decorator.py", line 82, in new_function
    return function(*args, **kwargs)
  File "<ipython-input-31-864ac19e3ab7>", line 6, in execute_function_and_get_output
    output = my_func(path)
  File "<ipython-input-31-864ac19e3ab7>", line 23, in my_func
    result = ocr.ocr(img)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/paddleocr.py", line 555, in ocr
    dt_boxes, rec_res, _ = self.__call__(img, cls)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infer/predict_system.py", line 95, in __call__
    rec_res, elapse = self.text_recognizer(img_crop_list)
  File "/usr/local/lib/python3.10/dist-packages/paddleocr/tools/infe

[2023/07/22 10:32:33] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
	 2 Pieces (240 Pieces X 21g)
[2023/07/22 10:32:34] ppocr WARNING: Since the angle classifier is not initialized, the angle classifier will not be uesd during the forward process
9 Amulya Dairy Whitener - AR AP - 2pc (240pc 21g)


In [ ]:
times = np.array(times)
print('Average Time',np.mean(times))

Average Time 0:00:02.045071


In [ ]:
max(times)

datetime.timedelta(seconds=4, microseconds=777454)